In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append("../")

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [ ]:
import pandas as pd
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer

In [ ]:
# [pin]
file_path = "../data/data-hard.csv"
root_path = "../data/"


df = pd.read_csv(file_path)
df["prefix"] = "clsorg"
df = df.rename({"message": "input_text", "label": "target_text"}, axis=1)
df.sample(20)

,input_text,target_text,prefix
3755,​​Проблема затоваривания рынка недвижимости бу...,111-4;100-5,clsorg
6654,🗓КАЛЕНДАРЬ НА СЕГОДНЯ — 2023.07.24 🇦🇺Австрали...,53-3;231-3,clsorg
1587,#ROSN ROSN СД РОСНЕФТИ 10 НОЯБРЯ РАССМОТРИТ РЕ...,112-4,clsorg
7171,🧠 Закрытые идеи Мозгового центра Сигналов РЦБ ...,152-3;111-3;127-3;89-3;160-3;163-3;254-3;221-3...,clsorg
3678,"​​Индекс МосБиржи по итогам недели: +2,04% Ит...",100-3,clsorg
914,"""🇷🇺#YNDX """"Яндекс"""" объединит бизнес собствен...",236-3,clsorg
6814,🛢🇷🇺#спг #россия #NVTK СФ одобрил закон об эксп...,115-4,clsorg
1819,#стратегиятактика #колумнистика Минус 39 млрд...,150-2;7-2,clsorg
5913,💥🇷🇺#MAGN = +5% = макс за 1 год,90-3,clsorg
4607,⬆️ #YNDX 101.4M₽ +0.06% 🔷 Аномальный объем Y...,236-4,clsorg


In [ ]:
m_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(m_name)

In [ ]:
from src.t5.dataset import NERDataModel
from src.t5.model import NERModel

BATCH_SIZE = 64
EPOCHS = 10
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)
data_module = NERDataModel(train_df, test_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

In [ ]:
model = NERModel()

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="ner",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min",
)

trainer = Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=EPOCHS,
    accelerator="cuda",
)

In [ ]:
!rm -r lightning_logs

In [ ]:
trainer.fit(model, data_module)

In [ ]:
trained_model = NERModel.load_from_checkpoint("checkpoints/ner-v5.ckpt")
trained_model.freeze()

In [ ]:
from src.t5.utils import evaluate_metric, generate_answer_batched

predictions = generate_answer_batched(
    trained_model=trained_model, tokenizer=tokenizer, data=test_df, batch_size=64
)

In [ ]:
ldf = test_df.copy()
ldf["predictions"] = predictions
ldf[["tcomp", "tsent"]] = (
    ldf["target_text"].str.split(";", expand=True)[0].str.split("-", expand=True)
)
ldf[["pcomp", "psent"]] = (
    ldf["predictions"].str.split(";", expand=True)[0].str.split("-", expand=True)
)

In [ ]:
# [pin]

evaluate_metric(
    company_predictions=ldf["pcomp"].tolist(),
    company_labels=ldf["tcomp"].tolist(),
    sentiment_predictions=ldf["psent"].tolist(),
    sentiment_labels=ldf["tsent"].tolist(),
)

{'total': 57.80824034396828,
 'f1': 0.6391920745477017,
 'accuracy': 0.5169727323316639}